In [15]:
import os
import docx2txt
import PyPDF2
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [31]:
import os

os.makedirs("job_description", exist_ok=True)

with open("job_description/job_description.txt", "w", encoding="utf-8") as f:
    f.write("We are looking for a Python developer with experience in Pandas, Flask, and ML.")
    with open("job_description/job_description.txt", "r", encoding="utf-8") as file:
      jd_text = file.read()

In [18]:
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            if page.extract_text():
                text += page.extract_text()
    return text

def extract_text_from_docx(docx_path):
    return docx2txt.process(docx_path)

resume_texts = []
file_names = []

for file in os.listdir("resumes"):
    path = os.path.join("resumes", file)
    if file.endswith(".pdf"):
        text = extract_text_from_pdf(path)
    elif file.endswith(".docx"):
        text = extract_text_from_docx(path)
    elif file.endswith(".txt"):
        with open(path, "r", encoding="utf-8") as f:
            text = f.read()
    else:
        continue
    resume_texts.append(text)
    file_names.append(file)

In [33]:
all_docs = [jd_text] + resume_texts
vectorizer = TfidfVectorizer(stop_words="english")
vectors = vectorizer.fit_transform(all_docs)

scores = cosine_similarity(vectors[0:1], vectors[1:]).flatten()
ranked_resumes = sorted(zip(file_names, resume_texts, scores), key=lambda x: x[2], reverse=True)

In [34]:
top_resume = ranked_resumes[0]
print("✅ Top Resume:", top_resume[0])
print("⭐ Relevance Score:", round(top_resume[2], 2))

jd_keywords = set(jd_text.lower().split())
resume_keywords = set(top_resume[1].lower().split())
matched_keywords = jd_keywords.intersection(resume_keywords)

print("\n📌 Reason for Selection:")
print(f"✓ {len(matched_keywords)} matching keywords found")
print("→", ", ".join(list(matched_keywords)[:20]), "...")

✅ Top Resume: Balaabirami Content Writer SEO-optimized .pdf
⭐ Relevance Score: 0.0

📌 Reason for Selection:
✓ 0 matching keywords found
→  ...


In [35]:
print("Resumes loaded:", len(resume_texts))
print("JD text length:", len(jd_text))

Resumes loaded: 12
JD text length: 0
